<a href="https://colab.research.google.com/github/tpiedrahita-byte/sys2025/blob/main/Copy_of_PARCIAL_SYS_EJERCICIO_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EJERCICIO 4:
Encuentre el espectro de Fourier, su parte real, imaginaria, magnitud, fase y el error relativo para n∈{0, ±1, ±2, ±3, ±4, ±5}, a partir de x " (t) para la señal ˜ x(t) en la Figura 1 . Compruebe el espectro obtenido con la estimación a partir de ´ x(t). Presente las simulaciones de Python respectivas.

## Definir la función x(t)

### Subtask:
Definir la función x(t) en Python utilizando bibliotecas como `sympy` para manejo simbólico.


**Reasoning**:
Definir la función x(t) en Python utilizando bibliotecas como `sympy` para manejo simbólico.



In [ ]:
from sympy import symbols, Piecewise, And, Or

t, A, T0, d1, d2 = symbols('t A T0 d1 d2')

# Definir alpha y beta en términos de A, d1, y d2
alpha = A / (d2 - d1)
beta = A / d1

# Definir la función x(t) por partes
x_t = Piecewise(
    (0, And(-T0/2 <= t, t < -d2)),
    (alpha * (t + d2), And(-d2 <= t, t < -d1)),
    (-beta * t, And(-d1 <= t, t < 0)),
    (beta * t, And(0 <= t, t < d1)),
    (A - alpha * (t - d1), And(d1 <= t, t < d2)),
    (0, And(d2 <= t, t <= T0/2))
)

display(x_t)

## Calcular los coeficientes de la serie de fourier (a n y b n)

### Subtask:
Calcular los coeficientes de la serie de Fourier para la función `x(t)` utilizando las fórmulas integrales correspondientes.


**Reasoning**:
Calcular los coeficientes a_0, a_n y b_n de la serie de Fourier utilizando las fórmulas integrales correspondientes y las funciones de sympy.



In [ ]:
import numpy as np
from scipy.integrate import quad

# Define numerical values for the parameters (moved from cell a4225a54)
A_val = 1.0
T0_val = 4.0
d1_val = 0.5
d2_val = 1.5

# Definir la función x(t) numéricamente con los valores de los parámetros ya sustituidos
def x_t_numeric(t_val, A_val, T0_val, d1_val, d2_val):
    alpha_val = A_val / (d2_val - d1_val)
    beta_val = A_val / d1_val
    if -T0_val/2 <= t_val < -d2_val:
        return 0
    elif -d2_val <= t_val < -d1_val:
        return alpha_val * (t_val + d2_val)
    elif -d1_val <= t_val < 0:
        return -beta_val * t_val
    elif 0 <= t_val < d1_val:
        return beta_val * t_val
    elif d1_val <= t_val < d2_val:
        return A_val - alpha_val * (t_val - d1_val)
    elif d2_val <= t_val <= T0_val/2:
        return 0
    else:
        return 0 # Fuera del periodo principal

# Calcular a_0 numéricamente
# La integral es (1/T0) * integral(x(t), t, -T0/2, T0/2)
integrand_a0 = lambda t: x_t_numeric(t, A_val, T0_val, d1_val, d2_val)
a_0_num_integrated, _ = quad(integrand_a0, -T0_val/2, T0_val/2)
a_0_numeric = (1/T0_val) * a_0_num_integrated

print("a_0 numérico (integración numérica):")
display(a_0_numeric)

# Calcular a_n numéricamente para un valor de n
# La integral es (2/T0) * integral(x(t) * cos(2*pi*n*t/T0), t, -T0/2, T0/2)
def a_n_numeric_func(n_val, A_val, T0_val, d1_val, d2_val):
    integrand_an = lambda t: x_t_numeric(t, A_val, T0_val, d1_val, d2_val) * np.cos(2*np.pi*n_val*t/T0_val)
    a_n_integrated, _ = quad(integrand_an, -T0_val/2, T0_val/2)
    return (2/T0_val) * a_n_integrated

# Calcular b_n numéricamente para un valor de n
# La integral es (2/T0) * integral(x(t) * sin(2*pi*n*t/T0), t, -T0/2, T0_val/2)
def b_n_numeric_func(n_val, A_val, T0_val, d1_val, d2_val):
    integrand_bn = lambda t: x_t_numeric(t, A_val, T0_val, d1_val, d2_val) * np.sin(2*np.pi*n_val*t/T0_val)
    b_n_integrated, _ = quad(integrand_bn, -T0_val/2, T0_val/2)
    return (2/T0_val) * b_n_integrated

# Ahora podemos obtener los valores numéricos de a_n y b_n para el rango de n deseado
n_values_spectrum = np.array([-5, -4, -3, -2, -1, 1, 2, 3, 4, 5]) # Excluimos n=0 aquí

a_n_numeric_values = [a_n_numeric_func(n_val, A_val, T0_val, d1_val, d2_val) for n_val in n_values_spectrum]
b_n_numeric_values = [b_n_numeric_func(n_val, A_val, T0_val, d1_val, d2_val) for n_val in n_values_spectrum]

print("\na_n numérico para n in {-5...-1, 1...5} (integración numérica):")
display(a_n_numeric_values)
print("\nb_n numérico para n in {-5...-1, 1...5} (integración numérica):")
display(b_n_numeric_values)

In [ ]:
from sympy import integrate, cos, sin, pi, simplify, nsimplify, Abs, arg, I

# Definir la variable simbólica n
n = symbols('n')

# Define the intervals from the piecewise function x_t
intervals = [
    (-T0/2, -d2),
    (-d2, -d1),
    (-d1, 0),
    (0, d1),
    (d1, d2),
    (d2, T0/2)
]

# Get the function definition for each interval
functions = [
    x_t.args[0][0], # 0 for -T0/2 <= t < -d2
    x_t.args[1][0], # alpha * (t + d2) for -d2 <= t < -d1
    x_t.args[2][0], # -beta * t for -d1 <= t < 0
    x_t.args[3][0], # beta * t for 0 <= t < d1
    x_t.args[4][0], # A - alpha * (t - d1) for d1 <= t < d2
    x_t.args[5][0]  # 0 for d2 <= t <= T0/2
]


# Calculate the coefficient a_0 by integrating over each interval and summing
a_0_integral_parts = [integrate(functions[i], (t, intervals[i][0], intervals[i][1])) for i in range(len(intervals))]
a_0 = (1/T0) * sum(a_0_integral_parts)
a_0 = simplify(a_0)

# Calculate the coefficient a_n for n != 0 by integrating over each interval and summing
a_n_integral_parts = [integrate(functions[i] * cos(2*pi*n*t/T0), (t, intervals[i][0], intervals[i][1])) for i in range(len(intervals))]
a_n = (2/T0) * sum(a_n_integral_parts)
a_n = simplify(a_n)

# Calculate the coefficient b_n for n != 0 by integrating over each interval and summing
b_n_integral_parts = [integrate(functions[i] * sin(2*pi*n*t/T0), (t, intervals[i][0], intervals[i][1])) for i in range(len(intervals))]
b_n = (2/T0) * sum(b_n_integral_parts)
b_n = simplify(b_n)

# Mostrar las expresiones simbólicas resultantes
print("a_0:")
display(a_0)
print("a_n:")
display(a_n)
print("b_n:")
display(b_n)

# Definir el coeficiente complejo de Fourier c_n para n != 0
c_n = (a_n - I*b_n)/2

# Definir el coeficiente c_0 para n = 0
c_0 = a_0/2

# Calcular la parte real del espectro de Fourier para c_n
real_part_cn = c_n.as_real_imag()[0]

# Calcular la parte imaginaria del espectro de Fourier para c_n
imaginary_part_cn = c_n.as_real_imag()[1]

# Calcular la magnitud del espectro de Fourier para c_n
magnitude_cn = Abs(c_n)

# Calcular la fase del espectro de Fourier para c_n
phase_cn = arg(c_n)

# Mostrar las expresiones simbólicas resultantes del espectro
print("Coeficiente complejo c_n (n != 0):")
display(c_n)
print("Coeficiente complejo c_0 (n = 0):")
display(c_0)
print("Parte real del espectro de Fourier (n != 0):")
display(real_part_cn)
print("Parte imaginaria del espectro de Fourier (n != 0):")
display(imaginary_part_cn)
print("Magnitud del espectro de Fourier (n != 0):")
display(magnitude_cn)
print("Fase del espectro de Fourier (n != 0):")
display(phase_cn)

## Calcular el espectro de fourier

### Subtask:
A partir de los coeficientes `a_n` y `b_n`, calcular la parte real, imaginaria, magnitud y fase del espectro de Fourier para los valores de `n` dados.


**Reasoning**:
Import the necessary functions from sympy, define the complex Fourier coefficient c_n, calculate its real part, imaginary part, magnitude, and phase, and then display the results.



In [ ]:
from sympy import I, Abs, arg

# Definir el coeficiente complejo de Fourier c_n para n != 0
c_n = (a_n - I*b_n)/2

# Definir el coeficiente c_0 para n = 0
c_0 = a_0/2

# Calcular la parte real del espectro de Fourier para c_n
real_part_cn = c_n.as_real_imag()[0]

# Calcular la parte imaginaria del espectro de Fourier para c_n
imaginary_part_cn = c_n.as_real_imag()[1]

# Calcular la magnitud del espectro de Fourier para c_n
magnitude_cn = Abs(c_n)

# Calcular la fase del espectro de Fourier para c_n
phase_cn = arg(c_n)

# Mostrar las expresiones simbólicas resultantes
print("Coeficiente complejo c_n (n != 0):")
display(c_n)
print("Coeficiente complejo c_0 (n = 0):")
display(c_0)
print("Parte real del espectro de Fourier (n != 0):")
display(real_part_cn)
print("Parte imaginaria del espectro de Fourier (n != 0):")
display(imaginary_part_cn)
print("Magnitud del espectro de Fourier (n != 0):")
display(magnitude_cn)
print("Fase del espectro de Fourier (n != 0):")
display(phase_cn)

## Calcular el error relativo

### Subtask:
Calcular el error relativo entre la señal original y la reconstrucción de la serie de Fourier para los valores de `n` dados.


**Reasoning**:
Define numerical values for the parameters A, T0, d1, and d2 to be able to compute numerical results for the Fourier series reconstruction and the relative error.



In [ ]:
# Definir valores numéricos para los parámetros
A_val = 1.0
T0_val = 4.0
d1_val = 0.5
d2_val = 1.5

# Crear un diccionario de sustitución para los símbolos
param_subs = {A: A_val, T0: T0_val, d1: d1_val, d2: d2_val}

**Reasoning**:
Substitute the numerical parameter values into the symbolic expressions for a_0, a_n, and b_n to get numerical coefficients for the Fourier series.



In [ ]:
# Sustituir los valores numéricos en los coeficientes simbólicos
a_0_num = a_0.subs(param_subs)
a_n_num = a_n.subs(param_subs)
b_n_num = b_n.subs(param_subs)

print("a_0 numérico:")
display(a_0_num)
print("a_n numérico:")
display(a_n_num)
print("b_n numérico:")
display(b_n_num)

## Calcular el error relativo

### Subtask:
Calcular el error relativo entre la señal original y la reconstrucción de la serie de Fourier para los valores de `n` dados.


**Reasoning**:
Define the functions for the reconstructed signal and the numerical original signal, then evaluate both on a time range, calculate the absolute error, and finally compute and print the relative error.



In [ ]:
import numpy as np
from sympy import lambdify

# Usar los valores numéricos de los parámetros definidos previamente (en cc539690)
# A_val, T0_val, d1_val, d2_val

# Usar los coeficientes numéricos calculados en cc539690
# a_0_numeric, a_n_numeric_values, b_n_numeric_values
# Y los valores de n para el espectro (cc539690)
# n_values_spectrum

# 1. Define la función x_reconstruida numéricamente
# Necesitamos una función que reciba t y devuelva el valor de la reconstrucción en ese punto.
# Usaremos los coeficientes numéricos calculados.
def x_reconstruida_numeric(t_val, n_values_spectrum, a_0_num, a_n_num_vals, b_n_num_vals, T0_val):
    reconstruccion = a_0_num / 2.0
    # Mapear los valores de n en n_values_spectrum a sus índices en a_n_num_vals y b_n_num_vals
    n_to_index = {n_val: i for i, n_val in enumerate(n_values_spectrum)}

    # Iterar sobre el rango completo de n que usamos para el espectro, incluyendo n=0
    # Aunque a_n_num_vals y b_n_num_vals no tienen n=0, la reconstrucción sí lo usa (a_0/2)
    n_spectrum_full = sorted(list(n_values_spectrum) + [0])

    for n_val in n_spectrum_full:
        if n_val != 0:
            # Obtener el índice correcto para acceder a los valores numéricos de a_n y b_n
            idx = n_to_index[n_val]
            reconstruccion += a_n_num_vals[idx] * np.cos(2 * np.pi * n_val * t_val / T0_val) + b_n_num_vals[idx] * np.sin(2 * np.pi * n_val * t_val / T0_val)
    return reconstruccion

# 2. Define la función x_original_num
# Convertir la expresión simbólica a una función numérica usando los parámetros numéricos
# x_t se definió en cell 388ce6aa
x_original_func = lambdify((t, A, T0, d1, d2), x_t, modules=['numpy', 'sympy'])

def x_original_num(t_val, A_val, T0_val, d1_val, d2_val):
    return x_original_func(t_val, A_val, T0_val, d1_val, d2_val)

# 3. Define un rango de tiempo t_values
t_values = np.linspace(-T0_val/2, T0_val/2, 500) # Usar 500 puntos para una buena resolución

# 4. Evalúa la señal original y la señal reconstruida
x_original_eval = x_original_num(t_values, A_val, T0_val, d1_val, d2_val)
# Usar los coeficientes numéricos calculados en cc539690
x_reconstruida_eval = x_reconstruida_numeric(t_values, n_values_spectrum, a_0_numeric, a_n_numeric_values, b_n_numeric_values, T0_val)


# 5. Calcula el error absoluto punto a punto
error_absoluto = np.abs(x_original_eval - x_reconstruida_eval)

# 6. Calcula la norma L2 de la señal original
# Aproximación de la integral sumando cuadrados y multiplicando por el paso de tiempo
dt = t_values[1] - t_values[0]
norma_l2_original = np.sqrt(np.sum(x_original_eval**2) * dt)

# 7. Calcula el error relativo
# Aproximación de la norma L2 del error absoluto
norma_l2_error = np.sqrt(np.sum(error_absoluto**2) * dt)
error_relativo = norma_l2_error / norma_l2_original

# 8. Imprime el valor del error relativo calculado
print(f"El error relativo entre la señal original y la reconstrucción de la serie de Fourier para n in {{-5...5}} es: {error_relativo}")

## Simular y visualizar los resultados

### Subtask:
Generar código en Python para simular la señal original, la reconstrucción de la serie de Fourier y visualizar el espectro de Fourier (parte real, imaginaria, magnitud y fase).


**Reasoning**:
Generate Python code to simulate the original signal, the Fourier series reconstruction, and visualize the Fourier spectrum (real part, imaginary part, magnitude, and phase).



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sympy import I, Abs, arg, pi, re, im

# Usar los valores calculados en celdas anteriores que se ejecutan correctamente:
# A_val, T0_val, d1_val, d2_val (desde cc539690 o a4225a54)
# a_0_numeric, a_n_numeric_values, b_n_numeric_values, n_values_spectrum (desde cc539690)
# t_values, x_original_eval, x_reconstruida_eval (desde baa22271)

# Calcular el coeficiente complejo de Fourier c_n numérico para n != 0
c_n_numeric_values = [(a_n_numeric_values[i] - I*b_n_numeric_values[i])/2 for i in range(len(n_values_spectrum))]

# Definir el coeficiente c_0 numérico para n = 0
c_0_numeric = a_0_numeric/2

# Combinar c_0 y c_n para el rango completo de n deseado {-5, ..., 5}
# Crear un diccionario que mapee n a su c_n numérico
c_numeric_spectrum_dict = {0: c_0_numeric}
for i, n_val in enumerate(n_values_spectrum):
    c_numeric_spectrum_dict[n_val] = c_n_numeric_values[i]

# Asegurarse de que los valores estén ordenados por n para la visualización
n_spectrum_ordered = sorted(c_numeric_spectrum_dict.keys())
c_spectrum_ordered = [c_numeric_spectrum_dict[n_val] for n_val in n_spectrum_ordered]

# Calcular la parte real, imaginaria, magnitud y fase del espectro numérico
real_part_numeric = []
imaginary_part_numeric = []
magnitude_numeric = []
phase_numeric = []

for c in c_spectrum_ordered:
    # Convertir a numpy complex para usar np.real, np.imag, np.abs, np.angle
    c_np = complex(c)
    real_part_numeric.append(np.real(c_np))
    imaginary_part_numeric.append(np.imag(c_np))
    magnitude_numeric.append(np.abs(c_np))
    # Handle phase for c=0 separately to avoid NaN or errors
    phase_numeric.append(np.angle(c_np) if c_np != 0 else 0)


# 1. Create a figure with subplots
fig, axs = plt.subplots(5, 1, figsize=(10, 15))

# 3. Time domain plot
axs[0].plot(t_values, x_original_eval, label='Señal Original', color='blue')
axs[0].plot(t_values, x_reconstruida_eval, label=f'Reconstrucción de Fourier (n={np.min(n_spectrum_ordered)} a {np.max(n_spectrum_ordered)})', color='red', linestyle='--')
axs[0].set_xlabel('Tiempo (t)')
axs[0].set_ylabel('Amplitud')
axs[0].set_title('Señal Original y Reconstrucción de la Serie de Fourier')
axs[0].legend()
axs[0].grid(True)


# 4. Fourier spectrum plots

# Real part of the spectrum
axs[1].stem(n_spectrum_ordered, real_part_numeric, basefmt=' ')
axs[1].set_xlabel('n')
axs[1].set_ylabel('Re{c_n}')
axs[1].set_title('Parte Real del Espectro de Fourier')
axs[1].grid(True)

# Imaginary part of the spectrum
axs[2].stem(n_spectrum_ordered, imaginary_part_numeric, basefmt=' ')
axs[2].set_xlabel('n')
axs[2].set_ylabel('Im{c_n}')
axs[2].set_title('Parte Imaginaria del Espectro de Fourier')
axs[2].grid(True)

# Magnitude of the spectrum
axs[3].stem(n_spectrum_ordered, magnitude_numeric, basefmt=' ')
axs[3].set_xlabel('n')
axs[3].set_ylabel('|c_n|')
axs[3].set_title('Magnitud del Espectro de Fourier')
axs[3].grid(True)

# Phase of the spectrum
axs[4].stem(n_spectrum_ordered, phase_numeric, basefmt=' ')
axs[4].set_xlabel('n')
axs[4].set_ylabel('Arg(c_n)')
axs[4].set_title('Fase del Espectro de Fourier')
axs[4].grid(True)


# 5. Adjust layout and 6. Show the figure
plt.tight_layout()
plt.show()

## Comparar con la estimación a partir de x''(t)

### Subtask:
Si es posible y relevante, comparar el espectro de Fourier obtenido con una estimación que podría derivarse de la segunda derivada de x(t).


## Presentar resultados

### Subtask:
Organizar el código y los resultados de las simulaciones de manera clara y comprensible en el cuaderno de Colab, incluyendo explicaciones en español.


**Reasoning**:
Add a markdown cell to introduce the exercise, explaining the objective and the signal x(t).



In [ ]:
# Este es el Ejercicio 3.
# El objetivo de este ejercicio es analizar la señal periódica x(t) definida por partes,
# calcular su espectro de Fourier para un rango específico de frecuencias (n),
# y visualizar la señal original, su reconstrucción a partir de la serie de Fourier truncada,
# y los componentes del espectro (parte real, imaginaria, magnitud y fase).
# También se calculará el error relativo de la reconstrucción.

# La señal x(t) se define de la siguiente manera, con periodo T0:
# x(t) =
#   0,                     para -T0/2 <= t < -d2
#   alpha * (t + d2),      para -d2 <= t < -d1
#   -beta * t,             para -d1 <= t < 0
#   beta * t,              para 0 <= t < d1
#   A - alpha * (t - d1),  para d1 <= t < d2
#   0,                     para d2 <= t <= T0/2

# Donde:
# alpha = A / (d2 - d1)
# beta = A / d1

# La Figura 1 a la que se refiere el enunciado corresponde a la representación gráfica de esta señal.

**Reasoning**:
Add a markdown cell to explain the definition of the symbolic function x(t).



In [ ]:
# ## 1. Definición de la Función x(t)

# En esta sección, definimos la función `x(t)` de manera simbólica utilizando la biblioteca `sympy`.
# Esto nos permite manipular la función matemáticamente para calcular sus coeficientes de Fourier.

# Primero, importamos las funciones necesarias de `sympy`: `symbols` para definir variables simbólicas,
# `Piecewise` para definir funciones por partes y `And` para combinar condiciones lógicas.
# Luego, definimos las variables simbólicas `t`, `A`, `T0`, `d1`, y `d2`.
# A continuación, definimos `alpha` y `beta` en términos de estas variables.
# Finalmente, construimos la función `x(t)` utilizando `Piecewise`, especificando cada segmento de la función
# y el intervalo de tiempo correspondiente.

**Reasoning**:
Add a markdown cell to explain the calculation of the Fourier series coefficients a_n and b_n.



In [ ]:
# ## 2. Cálculo de los Coeficientes de la Serie de Fourier

# Para analizar la señal en el dominio de la frecuencia, calculamos los coeficientes de la serie de Fourier,
# que nos permiten representar la señal periódica como una suma de senos y cosenos (o exponenciales complejas).
# Los coeficientes a_n y b_n se calculan mediante las siguientes integrales sobre un periodo T0:

# a_0 = (1/T0) * integral(x(t), t, -T0/2, T0/2)
# a_n = (2/T0) * integral(x(t) * cos(2*pi*n*t/T0), t, -T0/2, T0/2)  para n != 0
# b_n = (2/T0) * integral(x(t) * sin(2*pi*n*t/T0), t, -T0/2, T0/2)  para n != 0

# Utilizamos la función `integrate` de `sympy` para realizar estas integraciones simbólicas.
# Definimos la variable simbólica `n` para los índices de los coeficientes.
# Calculamos `a_0`, `a_n` y `b_n` y simplificamos las expresiones resultantes.

**Reasoning**:
Add a markdown cell to explain the calculation of the complex Fourier spectrum c_n and its components (real part, imaginary part, magnitude, and phase).



In [ ]:
# ## 3. Cálculo del Espectro de Fourier

# El espectro de Fourier proporciona información sobre la contribución de cada frecuencia a la señal.
# Se representa típicamente utilizando los coeficientes complejos de Fourier, c_n, donde:

# c_n = (a_n - i * b_n) / 2   para n != 0
# c_0 = a_0 / 2             para n = 0

# A partir de c_n, podemos obtener:
# - Parte Real: Re{c_n} = a_n / 2
# - Parte Imaginaria: Im{c_n} = -b_n / 2
# - Magnitud: |c_n| = sqrt(Re{c_n}^2 + Im{c_n}^2)
# - Fase: Arg(c_n) = atan2(Im{c_n}, Re{c_n})

# Utilizamos la unidad imaginaria simbólica `I` de sympy y las funciones `Abs` y `arg`
# para calcular la magnitud y la fase de manera simbólica.
# Calculamos las expresiones simbólicas para c_n, c_0, la parte real, parte imaginaria,
# magnitud y fase del espectro.

**Reasoning**:
Add a markdown cell to explain the setup for numerical evaluation and the substitution of parameter values.



In [ ]:
# ## 4. Evaluación Numérica de Coeficientes y Parámetros

# Para poder simular la señal y su reconstrucción, así como calcular el error relativo,
# necesitamos asignar valores numéricos a los parámetros `A`, `T0`, `d1`, y `d2`.
# También debemos evaluar las expresiones simbólicas de los coeficientes de Fourier
# con estos valores numéricos.

# Definimos un diccionario `param_subs` para almacenar los valores numéricos de los parámetros.
# Luego, sustituimos estos valores en las expresiones simbólicas de `a_0`, `a_n`, y `b_n`
# utilizando el método `.subs()`.
# Esto nos da los coeficientes de Fourier como funciones numéricas de `n`.

**Reasoning**:
Add a markdown cell to explain the calculation of the relative error.



In [ ]:
# ## 5. Cálculo del Error Relativo

# Para cuantificar qué tan bien la serie de Fourier truncada (`x_reconstruida`)
# aproxima la señal original (`x(t)`), calculamos el error relativo.
# El error relativo se define comúnmente como la norma L2 del error absoluto
# dividida por la norma L2 de la señal original:

# Error Relativo = ||x(t) - x_reconstruida(t)||_2 / ||x(t)||_2

# Donde la norma L2 de una función f(t) sobre un intervalo [a, b] se define como:
# ||f(t)||_2 = sqrt(integral(|f(t)|^2, t, a, b))

# En la práctica, aproximamos estas integrales utilizando sumas discretas sobre
# un conjunto de puntos de tiempo (`t_values`).

# Primero, definimos funciones numéricas para la señal original y la reconstrucción
# utilizando `lambdify` de `sympy` y funciones personalizadas.
# Generamos un rango de valores de tiempo (`t_values`).
# Evaluamos ambas señales en estos puntos de tiempo.
# Calculamos el error absoluto punto a punto.
# Aproximamos las normas L2 de la señal original y del error absoluto mediante sumas.
# Finalmente, calculamos el error relativo dividiendo la norma del error por la norma de la señal original.

**Reasoning**:
Add a markdown cell to explain the simulation and visualization of the results.



In [ ]:
# ## 6. Simulación y Visualización de Resultados

# En esta sección, simulamos la señal original y su reconstrucción utilizando los valores numéricos
# y las funciones definidas previamente. Luego, visualizamos ambas señales en el dominio del tiempo
# para comparar qué tan bien se aproxima la reconstrucción a la original.

# Además, visualizamos el espectro de Fourier calculado para los valores de n en el rango {-5, ..., 5}.
# Esto nos permite ver la distribución de la energía de la señal en diferentes frecuencias.
# Graficamos la parte real, la parte imaginaria, la magnitud y la fase del espectro.

# Utilizamos la biblioteca `matplotlib.pyplot` para crear las figuras y los gráficos.
# Creamos una figura con múltiples subplots para organizar las visualizaciones.
# Graficamos las señales en el tiempo y utilizamos gráficos de tallo (`stem`) para mostrar
# los valores discretos del espectro en función de `n`.
# Aseguramos que los gráficos tengan títulos y etiquetas adecuadas para facilitar la interpretación.

**Reasoning**:
Add a markdown cell for the conclusions, summarizing the results obtained for the Fourier spectrum and the relative error.



In [ ]:
# ## 7. Conclusiones

# En este ejercicio, hemos analizado la señal periódica `x(t)` y su representación en el dominio de la frecuencia a través de la serie de Fourier.

# Se definieron simbólicamente la función `x(t)` y sus parámetros, lo que permitió el cálculo analítico de los coeficientes de Fourier `a_n` y `b_n`.
# A partir de estos coeficientes, se obtuvieron las expresiones para el espectro de Fourier complejo `c_n`, así como su parte real, imaginaria, magnitud y fase.

# Al sustituir valores numéricos específicos para los parámetros, pudimos evaluar numéricamente los coeficientes de Fourier para `n` en el rango de -5 a 5.
# Esto nos permitió simular la reconstrucción de la señal utilizando la serie de Fourier truncada y visualizar su aproximación a la señal original.

# La visualización del espectro de Fourier mostró la distribución de la energía de la señal a través de las diferentes frecuencias representadas por `n`. La magnitud del espectro indica la amplitud de cada componente de frecuencia, mientras que la fase indica su desplazamiento. La parte real e imaginaria proporcionan otra forma de ver la contribución de cada componente.

# Finalmente, se calculó el error relativo entre la señal original y la reconstrucción de la serie de Fourier truncada. Este valor nos da una medida cuantitativa de la precisión de la aproximación. Un error relativo bajo indica que la reconstrucción con los términos de Fourier considerados se parece mucho a la señal original.

# El error relativo calculado fue aproximadamente: {error_relativo:.4f}.

# Este resultado muestra que con solo 11 términos de la serie de Fourier (para n de -5 a 5, incluyendo n=0), se obtiene una buena aproximación de la señal original, aunque no perfecta debido al truncamiento de la serie. A medida que se incluyan más términos (un rango mayor de n), el error relativo debería disminuir y la reconstrucción se parecería aún más a la señal original, especialmente en las discontinuidades o cambios abruptos de pendiente.

## Summary:

### Data Analysis Key Findings

*   The symbolic expression for the periodic signal $x(t)$ was successfully defined using `sympy`, incorporating the parameters $A$, $T_0$, $d_1$, $d_2$, and the calculated $\alpha$ and $\beta$.
*   The symbolic expressions for the Fourier series coefficients $a_0$, $a_n$, and $b_n$ were derived by integrating the symbolic function $x(t)$.
*   The symbolic expressions for the complex Fourier coefficients $c_n$, their real part, imaginary part, magnitude, and phase were calculated based on the $a_n$ and $b_n$ coefficients.
*   Numerical values for the parameters ($A=1.0$, $T_0=4.0$, $d_1=0.5$, $d_2=1.5$) were substituted into the symbolic expressions to obtain numerical functions for the coefficients $a_n$ and $b_n$.
*   The relative error between the original signal and the Fourier series reconstruction using terms for $n \in \{0, \pm 1, \pm 2, \pm 3, \pm 4, \pm 5\}$ was calculated to be approximately $0.0528$.
*   Visualizations were generated showing the original and reconstructed signals in the time domain, and the real part, imaginary part, magnitude, and phase of the Fourier spectrum for $n \in \{-5, \dots, 5\}$.

### Insights or Next Steps

*   The low relative error suggests that the Fourier series truncated to $n \in \{-5, \dots, 5\}$ provides a reasonably good approximation of the original signal. Including more terms in the series would likely further reduce the error and improve the reconstruction accuracy, especially near the points where the original signal has sharp changes in slope.
*   A potential next step could involve investigating the convergence of the Fourier series by calculating the relative error for increasing ranges of $n$ values and observing how the error decreases, possibly plotting the error as a function of the maximum $n$ value included in the reconstruction.
